In [1]:
import csv
import pandas as pd
import numpy as np
import pytz
from pyspark.sql import SparkSession
import os

In [ ]:
SPARK_HOME = os.path.abspath("G:\Other Games\spark-3.4.1-bin-hadoop3\spark-3.4.1-bin-hadoop3")

In [2]:
store_status_df = pd.read_csv("./assets/store status.csv")
open_hours_df = pd.read_csv("./assets/Menu hours.csv")
time_zone_df = pd.read_csv("./assets/bq-results-20230125-202210-1674678181880.csv")

In [21]:
len(time_zone_df)

13559

In [431]:
open_hours_df.head()
open_hours_df[open_hours_df['store_id'] == '8419537941919820732']

,store_id,day,start_time_local,end_time_local


In [432]:
time_zone_df.head()
time_zone_df[time_zone_df['store_id'] == '525217215046186225']

,store_id,timezone_str


In [3]:
result = pd.merge(store_status_df, open_hours_df, how="left", on="store_id")
result = pd.merge(result, time_zone_df, how='left', on="store_id")
len(result)

11823100

In [434]:
result.loc[result['timezone_str'].isna()]

,store_id,status,timestamp_utc,day,start_time_local,end_time_local,timezone_str
0,8419537941919820732,active,2023-01-22 12:09:39.388884 UTC,NaN,NaN,NaN,NaN
81,2057963137235512954,active,2023-01-21 17:15:29.921161 UTC,NaN,NaN,NaN,NaN
83,7815590034868021689,active,2023-01-21 17:15:39.285005 UTC,NaN,NaN,NaN,NaN
84,7493928216510270665,active,2023-01-21 17:15:43.777556 UTC,NaN,NaN,NaN,NaN
222,6656463669299127947,active,2023-01-25 17:01:33.08729 UTC,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
11820024,7562603514970763933,inactive,2023-01-22 03:00:53.021117 UTC,NaN,NaN,NaN,NaN
11820049,4658864072483512739,inactive,2023-01-19 14:09:05.647772 UTC,NaN,NaN,NaN,NaN
11820249,7758862105355689282,inactive,2023-01-25 11:02:18.389155 UTC,NaN,NaN,NaN,NaN
11820379,6231179318258751370,inactive,2023-01-19 08:04:58.142037 UTC,NaN,NaN,NaN,NaN


In [4]:
result['day'].fillna(6, inplace=True)
result['start_time_local'].fillna('00:00:00', inplace=True)
result['end_time_local'].fillna('23:59:59', inplace=True)
result['timezone_str'].fillna('America/Chicago', inplace=True)

In [28]:
# f = result.query('store_id == 7562603514970763933')
# f.query('status == "active"')
result.iloc[-10002300]

store_id                       2873839205725435068
status                                      active
timestamp_utc       2023-01-25 05:07:59.711411 UTC
day                                            0.0
start_time_local                          16:00:00
end_time_local                            21:00:00
timezone_str                      America/New_York
Name: 1820758, dtype: object

In [131]:
# convert string to timestamp
from datetime import datetime, timezone
something = store_status_df['timestamp_utc'][0].split(' ')
something.pop(-1)
time = ' '.join(something)
FORMAT = '%Y-%m-%d %H:%M:%S.%f'
ts = datetime.strptime(time, FORMAT).replace(tzinfo=timezone.utc).timestamp()
ts

1674389379.388884

In [611]:
import matplotlib.pyplot as plt
hello = result.query('store_id == 4715335257465089466 & day == 2').sort_values(by='timestamp_utc', ascending=True)
print(hello)
hello['timestamp_utc'] = hello.timestamp_utc.str.replace(' UTC', '')
hello['timestamp_utc'] = pd.to_datetime(hello['timestamp_utc'], format='%Y-%m-%d %H:%M:%S.%f')
ts = pd.Series(index=hello['timestamp_utc'])
ts = ts.resample('T').mean()
ts.interpolate(method='spline', order = 3)
len(ts)

                     store_id    status                   timestamp_utc  day  \
9813325   4715335257465089466  inactive    2023-01-18 19:50:54.2586 UTC  2.0   
9813334   4715335257465089466  inactive    2023-01-18 19:50:54.2586 UTC  2.0   
10682103  4715335257465089466  inactive  2023-01-18 20:30:36.077544 UTC  2.0   
10682112  4715335257465089466  inactive  2023-01-18 20:30:36.077544 UTC  2.0   
10450999  4715335257465089466  inactive   2023-01-18 21:10:07.73541 UTC  2.0   
...                       ...       ...                             ...  ...   
10244183  4715335257465089466  inactive  2023-01-25 16:12:25.133302 UTC  2.0   
9948768   4715335257465089466  inactive  2023-01-25 17:10:19.048211 UTC  2.0   
9948759   4715335257465089466  inactive  2023-01-25 17:10:19.048211 UTC  2.0   
9466483   4715335257465089466  inactive  2023-01-25 18:06:38.431991 UTC  2.0   
9466474   4715335257465089466  inactive  2023-01-25 18:06:38.431991 UTC  2.0   

         start_time_local end_time_loca

9977

In [612]:
ts = ts.reset_index()

In [325]:
hello = result.query('store_id == 8419537941919820732 & day == 7').sort_values(by='timestamp_utc', ascending=True)
hello['timestamp_utc'] = hello.timestamp_utc.str.replace(' UTC', '')
hello['timestamp_utc'] = pd.to_datetime(hello['timestamp_utc'], format='%Y-%m-%d %H:%M:%S.%f')
hello

,store_id,status,timestamp_utc,day,start_time_local,end_time_local,timezone_str
2570517,8419537941919820732,active,2023-01-18 20:00:34.808486,7.0,00:00:00,23:59:59,America/Chicago
2722926,8419537941919820732,active,2023-01-18 21:05:21.392845,7.0,00:00:00,23:59:59,America/Chicago
3081932,8419537941919820732,active,2023-01-18 22:02:47.288035,7.0,00:00:00,23:59:59,America/Chicago
5666773,8419537941919820732,active,2023-01-18 23:00:15.228605,7.0,00:00:00,23:59:59,America/Chicago
6176289,8419537941919820732,active,2023-01-19 00:02:06.425245,7.0,00:00:00,23:59:59,America/Chicago
...,...,...,...,...,...,...,...
757596,8419537941919820732,active,2023-01-25 14:03:11.846034,7.0,00:00:00,23:59:59,America/Chicago
901573,8419537941919820732,active,2023-01-25 15:05:02.050854,7.0,00:00:00,23:59:59,America/Chicago
7986325,8419537941919820732,active,2023-01-25 16:09:44.183230,7.0,00:00:00,23:59:59,America/Chicago
5103040,8419537941919820732,active,2023-01-25 17:13:13.759148,7.0,00:00:00,23:59:59,America/Chicago


In [613]:
## INTERPOLATION LOGIC

from io import StringIO
from csv import writer
import pandas as pd
import datetime

output = StringIO()
csv_writer = writer(output)
header = False
counter = 0

# for index, row in temp.iterrows():
#     if(not header):
#       csv_writer.writerow(temp)
#       header = True

#     csv_writer.writerow(row)
#     counter += 1
current_store_id = np.nan
current_status = np.nan
current_day = np.nan
current_start_time = np.nan
current_end_time = np.nan
current_timezone = np.nan

for value in ts["timestamp_utc"]:
    if not header:
        csv_writer.writerow(hello)
        header = True
    next_value = hello["timestamp_utc"].iat[counter]
    if value == next_value.replace(second=0, microsecond=0):
        current_store_id = hello["store_id"].iat[counter]
        current_status = hello["status"].iat[counter]
        current_day = hello["day"].iat[counter]
        current_start_time = hello["start_time_local"].iat[counter]
        current_end_time = hello["end_time_local"].iat[counter]
        current_timezone = hello["timezone_str"].iat[counter]
        # temp_dict = {
        #     "store_id": current_store_id,
        #     "status": current_status,
        #     "timestamp_utc": value,
        #     "day": current_day,
        #     "start_time_local": current_start_time,
        #     "end_time_local": current_end_time,
        #     "timezone_str": current_timezone,
        # }
        temp_list = [current_store_id, current_status, value, current_day, current_start_time, current_end_time, current_timezone]
        csv_writer.writerow(temp_list)
        counter += 1
    else:
        # temp_dict = {
        #     "store_id": current_store_id,
        #     "status": current_status,
        #     "timestamp_utc": value,
        #     "day": current_day,
        #     "start_time_local": current_start_time,
        #     "end_time_local": current_end_time,
        #     "timezone_str": current_timezone,
        # }
        temp_list = [current_store_id, current_status, value, current_day, current_start_time, current_end_time, current_timezone]
        csv_writer.writerow(temp_list)

output.seek(0)  # we need to get back to the start of the BytesIO
df = pd.read_csv(output)

In [614]:
df.loc[df['status'] == 'active']

,store_id,status,timestamp_utc,day,start_time_local,end_time_local,timezone_str


In [615]:
df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"], format="%Y-%m-%d %H:%M:%S")
df["start_time_local"] = pd.to_datetime(df["start_time_local"], format="%H:%M:%S")
df["end_time_local"] = pd.to_datetime(df["end_time_local"], format="%H:%M:%S")

In [621]:
# pytz giving incorrect conversions
import dateutil


def uptime_or_downtime_last_hour(uptime: bool = True):
    df["timestamp_utc"] = pd.to_datetime(
        df["timestamp_utc"], format="%Y-%m-%d %H:%M:%S"
    )
    df.sort_values("timestamp_utc", ascending=False, inplace=True)
    current_day = df["timestamp_utc"].iat[0]
    till_day = current_day - datetime.timedelta(hours=1)
    from_dates = df.loc[df["timestamp_utc"] > till_day]
    nearest_till_day = from_dates.iloc[-1]["timestamp_utc"]
    something = df.query("@nearest_till_day <= timestamp_utc <= @current_day")
    if uptime:
        return len(something[something["status"] == "active"])
    else:
        return len(something[something["status"] == "inactive"])


def uptime_or_downtime_last_day(uptime: bool = True):
    df.sort_values("timestamp_utc", ascending=False, inplace=True)
    current_day = df["timestamp_utc"].iat[0]
    till_day = current_day - datetime.timedelta(days=7)
    from_dates = df.loc[df["timestamp_utc"] > till_day]
    nearest_till_day = from_dates.iloc[-1]["timestamp_utc"]
    something = df.query("@nearest_till_day <= timestamp_utc <= @current_day")
    active_minutes = 0
    inactive_minutes = 0

    for index, row in something.iterrows():
        delta = False
        normal = False
        aware_timestamp = row["timestamp_utc"].replace(tzinfo=pytz.utc)
        week_day = aware_timestamp.weekday()
        active_week_day = row['day']
        aware_start_time = convert_to_utc(
            row["timezone_str"], row["start_time_local"], sample_date=aware_timestamp
        ).replace(
            year=aware_timestamp.year,
            month=aware_timestamp.month,
            day=aware_timestamp.day,
        )
        aware_end_time = convert_to_utc(
            row["timezone_str"], row["end_time_local"], sample_date=aware_timestamp
        ).replace(
            year=aware_timestamp.year,
            month=aware_timestamp.month,
            day=aware_timestamp.day,
        )
        if aware_end_time < aware_start_time:
            aware_end_time = aware_end_time + datetime.timedelta(days=1)
            normal = aware_start_time <= aware_timestamp <= aware_end_time
            if aware_end_time - aware_start_time == datetime.timedelta(
                days=0, hours=23, minutes=59, seconds=59
            ):
                aware_end_time = aware_end_time - datetime.timedelta(days=2)
                delta = aware_end_time <= aware_timestamp <= aware_start_time

        else:
            normal = aware_start_time <= aware_timestamp <= aware_end_time

        if row["status"] == "active" and week_day == active_week_day:
            if normal or delta:
                active_minutes += 1
        elif row["status"] == "inactive" and week_day == active_week_day:
            if normal or delta:
                inactive_minutes += 1

    print(active_minutes / (60))
    print(inactive_minutes / (60))
    print((active_minutes/(60*24)) + (inactive_minutes/(60*24)))
    return something
    # can't assume that all the active and inactive are continous
    # if uptime:
    #     active = something[something['status'] == "active"]
    #     if(len(active) != 0):
    #         return str(active['timestamp_utc'].iat[0] - active['timestamp_utc'].iat[-1]).split(" ")[-1]
    #     else:
    #         return '00:00:00'
    # else:
    #     inactive = something[something['status'] == "inactive"]
    #     if(len(inactive) != 0):
    #         return str(inactive['timestamp_utc'].iat[0] - inactive['timestamp_utc'].iat[-1]).split(" ")[-1]
    #     else:
    #         return '00:00:00'


def convert_to_utc(
    timezone_str: str, date: datetime.datetime, sample_date: datetime.datetime
) -> datetime.datetime:
    # sample_date is important as the result varies as the month changes
    date = date.replace(
        year=sample_date.year, day=sample_date.day, month=sample_date.month
    )
    tz = pytz.timezone(timezone_str)
    local_aware_time = tz.normalize(tz.localize(date, is_dst=False))
    as_utc = local_aware_time.astimezone(pytz.UTC)
    return as_utc


hello_there = uptime_or_downtime_last_day(uptime=False)

0.0
2.0166666666666666
0.08402777777777778


In [16]:
len(result)

11823058

In [8]:
df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"], format="%Y-%m-%d %H:%M:%S")
df["start_time_local"] = pd.to_datetime(df["start_time_local"], format="%H:%M:%S")
df["end_time_local"] = pd.to_datetime(df["end_time_local"], format="%H:%M:%S")

NameError: name 'df' is not defined

In [25]:
result.to_csv('./result.csv')
spark = SparkSession.builder.appName('hello').getOrCreate()
spark.read.option("header", True).csv("./result.csv").createOrReplaceTempView('result')

In [20]:
(spark.sql("SELECT * FROM result").show())

+---+-------------------+------+--------------------+----+----------------+--------------+-------------------+
|_c0|           store_id|status|       timestamp_utc| day|start_time_local|end_time_local|       timezone_str|
+---+-------------------+------+--------------------+----+----------------+--------------+-------------------+
|  0|8419537941919820732|active|2023-01-22 12:09:...|null|            null|          null|               null|
|  1|  54515546588432327|active|2023-01-24 09:06:...| 0.0|        08:00:00|      18:30:00|   America/New_York|
|  2|  54515546588432327|active|2023-01-24 09:06:...| 6.0|        08:00:00|      18:30:00|   America/New_York|
|  3|  54515546588432327|active|2023-01-24 09:06:...| 5.0|        08:00:00|      18:30:00|   America/New_York|
|  4|  54515546588432327|active|2023-01-24 09:06:...| 2.0|        08:00:00|      18:30:00|   America/New_York|
|  5|  54515546588432327|active|2023-01-24 09:06:...| 3.0|        08:00:00|      18:30:00|   America/New_York|
|

In [ ]:
import polars as pl


In [5]:
# FINALIZED

import datetime
from pyspark.sql.functions import udf, col, to_timestamp
from pyspark.sql.types import TimestampType, StringType


def convert_to_utc(
    timezone_str: str, date: datetime.datetime, sample_date: datetime.datetime
) -> datetime.datetime:
    # sample_date is important as the result varies as the month changes
    date = date.replace(
        year=sample_date.year, day=sample_date.day, month=sample_date.month
    )
    tz = pytz.timezone(timezone_str)
    local_aware_time = tz.normalize(tz.localize(date, is_dst=False))
    as_utc = local_aware_time.astimezone(pytz.UTC)
    return as_utc


# change in thinking
# lets just take one store right now
unique_store_ids = result["store_id"].unique()
print(len(unique_store_ids))

def replaceUTCStr(string: str):
    return string.replace(" UTC", "")


def replaceLocalTimeStr(string: str):
    return convert_to_utc(
        sample_timezone, datetime.datetime.strptime(string, "%H:%M:%S"), sample_date
    )


udf_replaceUtcStr = udf(replaceUTCStr, StringType())
udf_replaceLocalTimeStr = udf(replaceLocalTimeStr, TimestampType())
sample_date = None
sample_timezone = None

selected_store_id = 10913998916849586
# subset = spark.sql(f'SELECT * FROM result WHERE store_id == {selected_store_id}')
# sample_date = subset.first()['timestamp_utc']
# sample_timezone = subset.first()['timezone_str']
# subset.withColumn("timestamp_utc", udf_replaceUtcStr('timestamp_utc'))
# subset.select(col('timestamp_utc'), to_timestamp(col("timestamp_utc"), "yyyy-dd-MM HH24:mm:ss.SSSSSS")).show()
# spark_df = spark.createDataFrame(result)
hello = result.query("store_id == @selected_store_id").copy()
print(len(hello))
hello["timestamp_utc"] = hello["timestamp_utc"].str.replace(" UTC", "")
hello["timestamp_utc"] = pd.to_datetime(
    hello["timestamp_utc"], format="%Y-%m-%d %H:%M:%S.%f"
)
hello["day"] = hello["day"].astype(int)
hello["timestamp_utc"] = pd.to_datetime(
    hello["timestamp_utc"], format="%Y-%m-%d %H:%M:%S"
)
hello["start_time_local"] = pd.to_datetime(hello["start_time_local"], format="%H:%M:%S")
hello["end_time_local"] = pd.to_datetime(hello["end_time_local"], format="%H:%M:%S")
counter = 0
hello.sort_values("timestamp_utc", ascending=False, inplace=True)
sample_date = hello["timestamp_utc"].iat[0]
sample_timezone = hello["timezone_str"].iat[0]
hello["start_time_local"] = hello["start_time_local"].apply(
    lambda x: x.replace(
        year=sample_date.year, month=sample_date.month, day=sample_date.day
    )
)
hello["end_time_local"] = hello["end_time_local"].apply(
    lambda x: x.replace(
        year=sample_date.year, month=sample_date.month, day=sample_date.day
    )
)
# hello["start_time_local"] = (
#     hello["start_time_local"].dt.tz_localize(sample_timezone).dt.tz_convert(pytz.UTC)
# )
# hello["end_time_local"] = (
#     hello["end_time_local"].dt.tz_localize(sample_timezone).dt.tz_convert(pytz.UTC)
# )
for index, row in hello.iterrows():
    delta = False
    normal = False
    aware_timestamp = row["timestamp_utc"].replace(tzinfo=pytz.utc)
    aware_start_time = convert_to_utc(
        row["timezone_str"], row["start_time_local"], sample_date=aware_timestamp
    ).replace(
        year=aware_timestamp.year,
        month=aware_timestamp.month,
        day=aware_timestamp.day,
    )
    aware_end_time = convert_to_utc(
        row["timezone_str"], row["end_time_local"], sample_date=aware_timestamp
    ).replace(
        year=aware_timestamp.year,
        month=aware_timestamp.month,
        day=aware_timestamp.day,
    )
    if aware_end_time < aware_start_time:
        aware_end_time = aware_end_time + datetime.timedelta(days=1)
        normal = aware_start_time <= aware_timestamp <= aware_end_time
        if aware_end_time - aware_start_time == datetime.timedelta(
            days=0, hours=23, minutes=59, seconds=59
        ):
            aware_end_time = aware_end_time - datetime.timedelta(days=2)
            delta = aware_end_time <= aware_timestamp <= aware_start_time

    # print(row["timestamp_utc"].weekday(), row["day"])
    if row["timestamp_utc"].weekday() != row["day"] or (not delta and not normal):
        row.start_time_local = aware_start_time
        hello.drop(index, inplace=True)
    else:
        hello.loc[index, "start_time_local"] = aware_start_time
        hello.loc[index, "end_time_local"] = aware_end_time

    counter += 1


len(hello)

14092
132


7

In [17]:
for value in hello:
  print(value)

store_id
status
timestamp_utc
day
start_time_local
end_time_local
timezone_str


In [7]:
# FINALIZED

ts = pd.Series(index=hello['timestamp_utc'])
ts = ts.resample('T').mean()
ts.interpolate(method='spline', order = 3)
ts = ts.reset_index()
hello.sort_values('timestamp_utc', ascending=True, inplace=True)
ts

,timestamp_utc,0
0,2023-01-22 03:05:00,NaN
1,2023-01-22 03:06:00,NaN
2,2023-01-22 03:07:00,NaN
3,2023-01-22 03:08:00,NaN
4,2023-01-22 03:09:00,NaN
...,...,...
600,2023-01-22 13:05:00,NaN
601,2023-01-22 13:06:00,NaN
602,2023-01-22 13:07:00,NaN
603,2023-01-22 13:08:00,NaN


In [8]:
# FINALIZED

from io import StringIO
from csv import writer
import pandas as pd
import datetime

output = StringIO()
csv_writer = writer(output)
header = False
counter = 0

# for index, row in temp.iterrows():
#     if(not header):
#       csv_writer.writerow(temp)
#       header = True

#     csv_writer.writerow(row)
#     counter += 1
current_store_id = np.nan
current_status = np.nan
current_day = np.nan
current_start_time = np.nan
current_end_time = np.nan
current_timezone = np.nan

for value in ts["timestamp_utc"]:
    if not header:
        csv_writer.writerow(hello)
        header = True
    next_value = hello["timestamp_utc"].iat[counter]
    if value == next_value.replace(second=0, microsecond=0):
        current_store_id = hello["store_id"].iat[counter]
        current_status = hello["status"].iat[counter]
        current_day = hello["day"].iat[counter]
        current_start_time = hello["start_time_local"].iat[counter]
        current_end_time = hello["end_time_local"].iat[counter]
        current_timezone = hello["timezone_str"].iat[counter]
        # temp_dict = {
        #     "store_id": current_store_id,
        #     "status": current_status,
        #     "timestamp_utc": value,
        #     "day": current_day,
        #     "start_time_local": current_start_time,
        #     "end_time_local": current_end_time,
        #     "timezone_str": current_timezone,
        # }
        temp_list = [current_store_id, current_status, value, current_day, current_start_time, current_end_time, current_timezone]
        csv_writer.writerow(temp_list)
        counter += 1
    else:
        # temp_dict = {
        #     "store_id": current_store_id,
        #     "status": current_status,
        #     "timestamp_utc": value,
        #     "day": current_day,
        #     "start_time_local": current_start_time,
        #     "end_time_local": current_end_time,
        #     "timezone_str": current_timezone,
        # }
        temp_list = [current_store_id, current_status, value, current_day, current_start_time, current_end_time, current_timezone]
        csv_writer.writerow(temp_list)

output.seek(0)  # we need to get back to the start of the BytesIO
df = pd.read_csv(output)

In [9]:
print(len(df.query('status == "active"')))
print(len(df.query('status == "inactive"')))

4417
5654


In [9]:
# FINALIZED

df.sort_values("timestamp_utc", ascending=False, inplace=True)
df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"], format="%Y-%m-%d %H:%M:%S")
# df
current_date = df['timestamp_utc'].iat[0]
till_date = current_date - datetime.timedelta(days=7)
from_dates = df.loc[df["timestamp_utc"] > till_date]
nearest_till_date = from_dates.iloc[-1]['timestamp_utc']
required_df = df.query('@nearest_till_date <= timestamp_utc <= @current_date')
print(len(required_df.query('status == "active"')) / (60 * 24))
print(len(required_df.query('status == "inactive"')) / (60))

0.4201388888888889
0.0
